In [1]:
# se importa el conector de snowflake
import snowflake.connector 
# se importa del modulo de swnoflake la herramienta de pandas para ingestar un dataframe a una tabla de snowflake
from snowflake.connector.pandas_tools import write_pandas 
import pandas as pd

c:\Users\Jhovany Lara\AppData\Local\Programs\Python\Python310\lib\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
#se crea conexión
conn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws')
    #warehouse='PRUEBA',
    #role ='ACCOUNTADMIN',
    #schema ='PUBLIC')

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor() # se inicializa la conexión Creates a cursor object. Each statement will be executed in a new cursor object.
    cursor.execute(query)
    cursor.close()

In [ ]:
query = "Create database LAKE" # se inicializa en EV

execute_query(conn, query)

In [5]:
query = "Use database LAKE" # se inicializa en database prueba

execute_query(conn, query)

In [ ]:
query = "Create warehouse DW_EV" # se inicializa en database prueba

execute_query(conn, query)

In [6]:
query = "Use warehouse DW_EV" # se inicializa en database prueba

execute_query(conn, query)

In [ ]:
viewquery ="""
    create or replace TABLE CONTINENTE (
	ID_CONTINENTE INTEGER NOT NULL,
	CONTINENTE VARCHAR(20) NOT NULL,
	constraint PKEY_1 primary key (ID_CONTINENTE)
    );""" # se crea la tabla CONTINENTE

execute_query(viewquery)

In [24]:
viewquery ="""
    DROP TABLE INCOME;
    """ # se borra la tabla INCOME

execute_query(conn, viewquery)

In [25]:
viewquery ="""
    create or replace TABLE INCOME (
	ID_INCOME VARCHAR(7) NOT NULL,
	INCOME VARCHAR(30),
	constraint PKEY_1 primary key (ID_INCOME)
    );""" # se crea la tabla INCOME

execute_query(conn, viewquery)

In [ ]:
viewquery ="""
    create or replace TABLE INDICADOR (
	ID_INDICADOR VARCHAR(20) NOT NULL,
	CODIGO VARCHAR(20) NOT NULL,
	DESCRIPCION VARCHAR(16777216),
	constraint PKEY_1 primary key (ID_INDICADOR)
    );""" # se crea la tabla INDICADOR

execute_query(viewquery)

In [ ]:
viewquery ="""
    create or replace TABLE PAIS (
	ID_PAIS VARCHAR(3) NOT NULL,
	NOMBRE VARCHAR(30) NOT NULL,
	constraint PKEY_1 primary key (ID_PAIS)
    );""" # se crea la tabla PAIS

execute_query(viewquery)

In [ ]:
viewquery ="""
    create or replace TABLE EV (
	ID_PAIS VARCHAR(3) NOT NULL,
	ID_CONTINENTE INTEGER NOT NULL,
	ID_INDICADOR VARCHAR(20) NOT NULL,
	ID_INCOME VARCHAR(7) NOT NULL,
	ANIO INTEGER NOT NULL,
	constraint FKEY_1 foreign key (ID_PAIS) references PAIS(ID_PAIS),
	constraint FKEY_2 foreign key (ID_CONTINENTE) references CONTINENTE(ID_CONTINENTE),
	constraint FKEY_3 foreign key (ID_INDICADOR) references INDICADOR(ID_INDICADOR),
	constraint FKEY_4 foreign key (ID_INCOME) references INCOME(ID_INCOME)
    );""" # se crea la tabla de hechos EV

execute_query(viewquery)

In [ ]:
df = pd.DataFrame([(1,'America'), (2,'Europa'), (3,'Asia'),(4,'Africa'),(5,'Oceania')], columns=['ID_CONTINENTE', 'CONTINENTE']) # se crea dataframe con pandas
write_pandas(conn, df, 'CONTINENTE') # se ingesta el dataframe pd a la tabla costumers



In [ ]:
Nation = ['United States', 'Canada','Mexico','Costa Rica','Panama','Brazil','Argentina','Chile','Uruguay','Bolivia','Peru','Egypt, Arab Rep.','Libya','South Africa','Nigeria','Morocco','Australia','China','India','Thailand','Japan','Korea, Rep.','Israel','Saudi Arabia','Malaysia','Indonesia','Russian Federation','Turkiye','Spain','Bulgaria','France','Italy','Germany','United Kingdom','Norway','Sweden','Greece']
Nation_code= ['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA','MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR','ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
df = pd.DataFrame({'ID_PAIS',Nation_code, 'NOMBRE', Nation}) # se crea dataframe con pandas
write_pandas(conn, df, 'PAIS') # se ingesta el dataframe pd a la tabla costumers



In [26]:
df = pd.DataFrame([('H','Alto ingreso'), ('M','Ingreso medio alto'), ('LM','Ingreso medio bajo'),('L','Ingreso bajo')], columns=['ID_INCOME', 'INCOME']) # se crea dataframe con pandas
write_pandas(conn, df, 'INCOME') # se ingesta el dataframe pd a la tabla costumers
# se cortan las conexiones
conn.close()